In [1]:
import torch
from custom_transformer import CustomTransformer

In [2]:
torch.manual_seed(1234)

In [3]:
# Testing on random input text before actual python scripts to make sure everything works.
with (open("../data/input.txt", "r") as file):
    text = file.read()

vocab = sorted(set(text))
print(len(vocab))

65


In [4]:
ctoi = {ch : i for i, ch in enumerate(vocab)}
itos = {i : ch for i, ch in enumerate(vocab)}

encode = lambda s : [ctoi[c] for c in s]
decode = lambda s : ''.join([itos[i] for i in s])

In [5]:
data = encode(text)
train_size = int(len(data) * .9)
train = data[:train_size]
test = data[train_size:]

def get_batches(dataset, batches, context_len):
    pos = torch.randint(len(dataset) - context_len, (batches,))
    x = torch.stack([torch.tensor(dataset[x : x + context_len]) for x in pos])
    y = torch.stack([torch.tensor(dataset[x + 1 : x + context_len + 1]) for x in pos])

    return x,y

In [6]:
transformer = CustomTransformer(len(vocab), 10, 20, 4, 2)
optimizer = torch.optim.Adam(transformer.parameters(), lr=1e-3)

In [8]:
for i in range(10):
    train_in, train_target = get_batches(train, 4, 10)
    logits, loss = transformer(train_in, train_target)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
#    print(loss.item())

-18.236251831054688
-18.935283660888672
-19.42620277404785
-20.876977920532227
-22.171865463256836
-28.083812713623047
-15.307360649108887
-20.224943161010742
-26.409128189086914
-21.114221572875977
